## Import

In [1]:
import pandas as pd
import numpy as np
import csv
import sklearn.cross_validation as cv
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

## Fonction de création du fichier csv pour la soubmission
### La fonction n'est pas utilisée finalement

In [2]:
def create_submission(filename, expected): #expected.shape = (717625, 2)
    fic = open(filename,"wb")
    c=csv.writer(fic)
    c.writerow(("Id","Expected"))
    for i in range(len(expected)):
        c.writerow((np.int(expected[i,0]),np.max([0,expected[i,1]])))
    fic.close()

## Chargement des fichiers

In [3]:
#Path
pathTrain="C:/Users/Maxence/Documents/M2/FDMS/HMDIR/train.csv"
pathTest="C:/Users/Maxence/Documents/M2/FDMS/HMDIR/test.csv"
outfile="C:/Users/Maxence/Documents/M2/FDMS/HMDIR/solution.csv"
#Chargement des fichiers
train = pd.read_csv(pathTrain)
test = pd.read_csv(pathTest)
#On sépare en X et y train/test et on ne prend pas la colonne Id
Xtrain = train.iloc[:,1:train.columns.size-1]
ytrain = train.iloc[:,train.columns.size-1:]
Xtest = test.iloc[:,1:test.columns.size]
#Recupération des id de test pour la parti soumbission
ids = np.array([test['Id'].unique()])

## Features engenering

### Nous mettons simplement des 0 à la place des nan

In [4]:
Xtrain = Xtrain.fillna(0)
Xtest = Xtest.fillna(0)

In [16]:
Xtrain.head(50)
#ytrain[Xtrain.iloc[:,2:].sum(axis=1)==0]

,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,...,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th
0,3,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
1,16,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
2,25,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
3,35,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
4,45,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
5,55,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
6,1,2,9.0,5.0,7.5,10.5,15.0,10.5,16.5,23.5,...,0.998333,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998
7,6,2,26.5,22.5,25.5,31.5,26.5,26.5,28.5,32.0,...,0.998333,1.005000,0.0625,-0.1875,0.2500,0.6875,0.000000,0.000000,0.000000,1.409988
8,11,2,21.5,15.5,20.5,25.0,26.5,23.5,25.0,27.0,...,0.998333,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,0.000000,-0.350006,1.759994
9,16,2,18.0,14.0,17.5,21.0,20.5,18.0,20.5,23.0,...,0.998333,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998


## Apprentissage

### Simple regression Lineaire

In [26]:
#Apprentissage
regr = linear_model.LinearRegression()
regr.fit(Xtrain, ytrain)

In [29]:
#Prédiction
pred = regr.predict(Xtest)
#cv.cross_val_score(regr, Xtrain,ytrain, cv=10)
#pred = regr.predict(Xtest)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

### Random Forest

In [ ]:
#Apprentissage
rand_for = RandomForestClassifier(n_estimators=2)
rand_for.fit(Xtrain, ytrain)

In [ ]:
#Prediction
pred = rand_for.predict(Xtest)

## Création du fichier resultat

In [ ]:
#On ajoute la colonne de prédiction au fichier test
test['Expected']=pred
#On récupère Id et Expected puis on fait la moyenne des expected par id à l'aide d'un groupby
expectation = test[['Id','Expected']]
expectation = expectation.groupby('Id').mean()
#On met à 0 les valeurs négatives s'il y en a
expectation[expectation['Expected']<0]=0
#On crée le fichier du resultat
expectation.to_csv("previsions.csv",header=True)

'''
#Peu utile si on utlise la fonction to_csv juste au dessus

#Création du tableau final
expected=np.zeros((ids.shape[1],2))
expected[:,0]=ids
expected[:,1]=expectation['Expected']
create_submission("previsions.csv", expected)
'''